In [1]:
!pip install bitsandbytes
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.9 MB/s eta 0:00:00:00:0100:01


In [2]:
import pandas as pd
import numpy as np
import torch
import re
import string
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import time
import os
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from transformers import LlamaForCausalLM, GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.util import ngrams


In [3]:
pip install --upgrade pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 8.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pinecone import Pinecone
api_key = 'df47aa2a-f42c-4446-85bc-e2a4af03f79d'
pc = Pinecone(api_key=api_key)

In [5]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [6]:
index_name_1 = "hydro-question-answer"
index_name_2 = "hydro-chunk"

In [7]:
if index_name_1 not in pc.list_indexes().names():
    pc.create_index(
        index_name_1,
        dimension=768,  
        metric='cosine',
        spec=spec
    )
    while not pc.describe_index(index_name_1).status['ready']:
        time.sleep(1)

index_1 = pc.Index(index_name_1)
print("Question Index Stats:")
print(index_1.describe_index_stats())


Question Index Stats:
{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5716}},
 'total_vector_count': 5716,
 'vector_type': 'dense'}


In [8]:
if index_name_2 not in pc.list_indexes().names():
    pc.create_index(
        index_name_2,
        dimension=768,  
        metric='cosine',
        spec=spec
    )
    while not pc.describe_index(index_name_2).status['ready']:
        time.sleep(1)
index_2 = pc.Index(index_name_2)
print("Chunk Index Stats:")
print(index_2.describe_index_stats())

Chunk Index Stats:
{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 151}},
 'total_vector_count': 151,
 'vector_type': 'dense'}


In [9]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [11]:
model_name = "1MK26/llama3_FT"
model = LlamaForCausalLM.from_pretrained(model_name).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

adapter_config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

In [12]:
def query_pinecone(query, top_k):
    xq = retriever.encode([query]).tolist()
    xc_qa = index_1.query(vector=xq, top_k=top_k, include_metadata=True)
    xc_chunk = index_2.query(vector=xq, top_k=top_k, include_metadata=True)
    return xc_qa, xc_chunk

In [24]:
import re
def remove_questions_from_text(text):
    cleaned_text = re.sub(r'\s*\?.*$', '', text)  
    cleaned_text = re.sub(r'(How|What|Why|Where|When|Who)\s+.*\?$', '', cleaned_text)  
    cleaned_text = re.sub(r'\b(?:How|What|Why|Where|When|Who)\b.*\?', '', cleaned_text)
    return cleaned_text.strip()  

query1 = "What is the use of nanomaterials in hydrogen production?"
qa_result, chunk_result = query_pinecone(query1, top_k=1)

# Process and print results
print("QA Results:")
for match in qa_result['matches']:
    # Clean the retrieved content
    cleaned_answer = remove_questions_from_text(match['metadata'].get('generated_answer', ''))
    print(f"\nScore: {match['score']}, Metadata: {cleaned_answer}")

print("\nChunk Results:")
for match in chunk_result['matches']:
    # Clean the retrieved content
    cleaned_chunk = remove_questions_from_text(match['metadata'].get('Chunk', ''))
    print(f"\nScore: {match['score']}, Metadata: {cleaned_chunk}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

QA Results:

Score: 0.784365177, Metadata: Nanomaterials can help us generate hydrogen more efficiently in a number of ways, including: Catalysis: Nanomaterials can be used as catalysts for a variety of chemical reactions, including the reactions that produce hydrogen. By using nanomaterials as catalysts, we can increase the efficiency of these reactions and produce hydrogen more quickly and easily. Photocatalysis: Nanomaterials can also be used as photocatalysts, which are materials that use light to drive chemical reactions. Photocatalytic nanomaterials can be used to split water into hydrogen and oxygen, providing a clean and renewable source of hydrogen. Electrocatalysis: Nanomaterials can also be used as electrocatalysts, which are materials that use electricity to drive chemical reactions. Electrocatalytic nanomaterials can be used to produce hydrogen from water or other sources, using electricity as the energy source.In addition to these specific applications, nanomaterials can 

In [15]:
def generate_model_input(query_and_docs, max_length):  
    return tokenizer(
        query_and_docs,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
        max_length=max_length  
    )

def generate_model_answer(retrieved_content, query, temperature, top_k, max_length, min_length):
    model_input = generate_model_input(retrieved_content, max_length)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  

    with torch.no_grad():  
        generated_answers_encoded = model.generate(
            input_ids=model_input["input_ids"].to(device),
            attention_mask=model_input["attention_mask"].to(device),
            min_length=min_length,
            max_new_tokens=max_length,  # Using max_new_tokens instead of max_length
            temperature=temperature,
            top_k=top_k,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2,
            top_p=0.9
        )

    decoded_answers = tokenizer.batch_decode(
        generated_answers_encoded,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )

    return decoded_answers if decoded_answers else ["Answer not generated"]


In [16]:
perplexity_model = GPT2LMHeadModel.from_pretrained('gpt2')
perplexity_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
def full_evaluation(query, answers):
    def compute_embeddings(text):
        embeddings = embed_model.encode(text, convert_to_tensor=True)
        return embeddings
    metrics = {
        'cosine_similarities': [],
        'perplexities': []
    }

    for answer in answers:
        query_embedding = compute_embeddings(query)
        answer_embedding = compute_embeddings(answer)
        similarity_score = cosine_similarity(query_embedding.cpu().numpy().reshape(1, -1),
                                             answer_embedding.cpu().numpy().reshape(1, -1))[0][0]
        inputs = perplexity_tokenizer(answer, return_tensors='pt')
        with torch.no_grad():
            loss = perplexity_model(**inputs, labels=inputs['input_ids'])[0]
        perplexity_score = torch.exp(loss).item()
        metrics['cosine_similarities'].append(similarity_score)
        metrics['perplexities'].append(perplexity_score)
    for i in range(len(answers)):
        print(f"\nEvaluation Metrics for Answer :")
        print(f"Cosine Similarity: {metrics['cosine_similarities'][i]:.4f}")
        print(f"Perplexity: {metrics['perplexities'][i]:.4f}")
    return metrics

In [21]:
def optimize_parameters(query1):
    temperature_values = [0.3, 0.5, 0.7, 1.0]
    top_k_values = [4, 6, 8, 10]
    max_length_values = [150 ,250, 500, 700]
    min_length_values = [25, 50, 75, 100]
    best_score = -np.inf
    best_params = {}
    best_answer = ""
    qa_result, chunk_result = query_pinecone(query1, top_k=2)
    qa_highest_match = max(qa_result['matches'], key=lambda x: x['score'], default=None)
    chunk_highest_match = max(chunk_result['matches'], key=lambda x: x['score'], default=None)

    if qa_highest_match and qa_highest_match['score'] >= 0.4:
        retrieved_content = qa_highest_match['metadata']['generated_answer']
    elif chunk_highest_match and chunk_highest_match['score'] >= 0.4:
        retrieved_content = chunk_highest_match['metadata']['Chunk']
    else:
        return "Answer not found", {}, 0, 0  
    
    # Parameter optimization loop
    for temp in temperature_values:
        for top_k in top_k_values:
            for max_len in max_length_values:
                for min_len in min_length_values:
                    generated_answers = generate_model_answer(
                        retrieved_content, 
                        query1, 
                        temperature=temp,
                        top_k=top_k,
                        max_length=max_len,
                        min_length=min_len
                    )
                    
                    if not generated_answers:
                        continue
                        
                    metrics = full_evaluation(query1, generated_answers)
                    current_cosine = max(metrics.get('cosine_similarities', [0]))
                    current_perplexity = min(metrics.get('perplexities', [float('inf')]))  
                    current_score = current_cosine - (current_perplexity / 100)  
                    print(f"\nParameters: temp={temp}, top_k={top_k}, max_len={max_len}, min_len={min_len}")
                    print(f"Cosine: {current_cosine:.3f}, Perplexity: {current_perplexity:.1f}")
                    print(f"Answer: {generated_answers[0]}")
                    if current_score > best_score:
                        best_score = current_score
                        best_params = {
                            'temperature': temp,
                            'top_k': top_k,
                            'max_length': max_len,
                            'min_length': min_len
                        }
                        best_answer = generated_answers[0]
    
    # Final evaluation
    if best_answer:
        best_metrics = full_evaluation(query1, [best_answer])
        best_cosine = max(best_metrics.get('cosine_similarities', [0]))
        best_perplexity = min(best_metrics.get('perplexities', [float('inf')]))
    else:
        best_cosine, best_perplexity = 0, float('inf')
    
    print("\n=FINAL RESULTS")
    print(f"Best Parameters: {best_params}")
    print(f"Cosine Similarity: {best_cosine:.3f}")
    print(f"Perplexity: {best_perplexity:.1f}")
    print(f"\nBest Answer:\n{best_answer}")
    
    return best_answer, best_params, best_cosine, best_perplexity

In [22]:
def main():
    query = "What are the potential risks associated with the large-scale use of nanomaterials in hydrogen production?"
    best_answer, best_params, best_cosine_similarity, best_perplexity = optimize_parameters(query)
    print("Final Selected Answer:\n", best_answer)
    print("Optimal Parameters:", best_params)

In [23]:
if __name__ == "__main__":
    main()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Evaluation Metrics for Answer :
Cosine Similarity: 0.7037
Perplexity: 15.1036

Parameters: temp=0.3, top_k=4, max_len=150, min_len=25
Cosine: 0.704, Perplexity: 15.1
Answer: The large-scale use of nanomaterials in hydrogen production poses several potential risks that need to be carefully considered and managed: Environmental risks: Engineered nanomaterials have unique physicochemical properties that may differ from their bulk counterparts, and their potential environmental impacts are not fully understood. Concerns exist about the release of nanomaterials into the environment during production, use, and disposal, and their potential effects on ecosystems and human health. Health and safety risks: Nanomaterials can interact with biological systems in complex ways, and their potential health effects need to be thoroughly evaluated. Inhalation, ingestion, or dermal exposure to nanomaterials may lead to adverse health effects, including inflammation, oxidative stress, and genotoxicity. T

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Evaluation Metrics for Answer :
Cosine Similarity: 0.7779
Perplexity: 8.6664

Parameters: temp=1.0, top_k=10, max_len=700, min_len=100
Cosine: 0.778, Perplexity: 8.7
Answer: The large-scale use of nanomaterials in hydrogen production poses several potential risks that need to be carefully considered and managed: Environmental risks: Engineered nanomaterials have unique physicochemical properties that may differ from their bulk counterparts, and their potential environmental impacts are not fully understood. Concerns exist about the release of nanomaterials into the environment during production, use, and disposal, and their potential effects on ecosystems and human health. Health and safety risks: Nanomaterials can interact with biological systems in complex ways, and their potential health effects need to be thoroughly evaluated. Inhalation, ingestion, or dermal exposure to nanomaterials may lead to adverse health effects, including inflammation, oxidative stress, and genotoxicity. O

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Evaluation Metrics for Answer :
Cosine Similarity: 0.7779
Perplexity: 8.6664

=FINAL RESULTS
Best Parameters: {'temperature': 0.3, 'top_k': 4, 'max_length': 500, 'min_length': 25}
Cosine Similarity: 0.778
Perplexity: 8.7

Best Answer:
The large-scale use of nanomaterials in hydrogen production poses several potential risks that need to be carefully considered and managed: Environmental risks: Engineered nanomaterials have unique physicochemical properties that may differ from their bulk counterparts, and their potential environmental impacts are not fully understood. Concerns exist about the release of nanomaterials into the environment during production, use, and disposal, and their potential effects on ecosystems and human health. Health and safety risks: Nanomaterials can interact with biological systems in complex ways, and their potential health effects need to be thoroughly evaluated. Inhalation, ingestion, or dermal exposure to nanomaterials may lead to adverse health effects, 

In [ ]:
    temperature_values = [0.3, 0.5, 0.7, 1.0]
    top_k_values = [4, 6, 8, 10]
    max_length_values = [150 ,250, 500, 700]
    min_length_values = [25, 50, 75, 100]